<a href="https://colab.research.google.com/github/elango-ela/Gemma-Fine-tuning-with-Data-Science-dataset/blob/main/My_Gemma_finetune_OWN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-science-qna-llm-fine-tuning:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4500171%2F7707625%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240830%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240830T065207Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2cb6d2bbc8842a1ade0f8e43c2f8be4c996041bbac7192b813402c5cb9dc766d596cb3a5259b0c310de099b6b5f80c35671ac627cce5f87881a6bf13e952c47430ee4085d1f0d0bdacce07fa67d4a6bfe3d2a79ba4adcbe704119f212ac602761ca6af245d691ae8e9a151c84127b6cafd346de21a395f710093d5cb15627b163dd26d85454fed6a808f01b936d319169b12665c0c218aca053a72e83ede7952e0a727b6df24d447ad0f76818efbbba979ec118cad1c409f57890c8ded3dd8526b165e740927003e2870930bd8ff1b88c4e489e39ca2a760eca29a692ef29806f30401e66e2cdb23f566aaa91da78c1ed5dcef373b73cd14fb12732123f87cd3,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240830%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240830T065207Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da8aefe86305ff9ce677e803436713e077119ff4b3fcf5b13c9ca303d5c719af17fc9c3de7e89df209c8979863bfe517c72d90e4254e1a8d2ef6b3008b4d8200d72332e919827d1e6186118699f51107f061dfdb9277315f6b88e577050cc62ac5b74c0cfed2cea5059a5db241d408dfbb1d25680ca97562593fd08e32046e8fa1b003cec54e3613b35919ab6036103fa503fac4e19f877b9c6f32ce00bf3e6f8762119731be2776ede92c74775fef7a28b919dee2bd96c0e24217b792a3dd5937258158cb4ae202efe2bbc486346f555dd5b5a9919b3dd33cbad801473a121100b24f96792935cf8faac8725c741ea1c7dd728c434ccdaab7a4a8f8714f7f955'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

In [ ]:
!pip install accelerate peft bitsandbytes trl
!pip install -U datasets
!pip -q install git+https://github.com/huggingface/transformers.git
!pip install -U "huggingface_hun[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring-parser 0.15
    Uninstalling docstring-parser-0.15:
      Successfully uninstalled docstring-parser-0.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.0 MB/s eta 0:00:00:00:0100:01
  Attempting 

In [ ]:
import pandas as pd
import os
import torch

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import TrainingArguments
from transformers import pipeline
from transformers import logging


In [ ]:
from peft import LoraConfig
from trl import SFTTrainer

from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict

In [ ]:
def create_inst(r):
    inst="""Below is an instruction thta describes a task.write a response thta appropriate
    ### Instruction:{}
    ### Response:{}""".format(r['Question'],r['Answer'])
    return inst

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/data-science-qna-llm-fine-tuning/datascienceQnA.csv')


In [ ]:
df['text']=df.apply(create_inst,axis=1)

In [ ]:
dataset=Dataset.from_pandas(df)

In [ ]:
base_model="google/gemma-2b"
mew_model="google/gemma-2b-DataScSpeacial"

In [ ]:
lora_config=LoraConfig(r=8,target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","up_proj"],
                                            task_type="CAUSAL_LM",)

bnb_config=BitsAndBytesConfig(
load_in_4bit=True,bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16)

In [ ]:
model_path='/kaggle/input/gemma/transformers/2b/2'
tokenizer=AutoTokenizer.from_pretrained(model_path)
#tokenizer.padding_side="right"

model=AutoModelForCausalLM.from_pretrained(model_path,quantization_config=bnb_config,device_map={"":0})

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text="what is deepracer?"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)

outputs=model.generate(**inputs,max_new_tokens=250)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

what is deepracer?

DeepRacer is a platform for building and racing autonomous racing cars.

How do I get started?

You can start by signing up for a free account.

Once you have signed up, you can start building your first car by following the instructions in the Getting Started Guide.

How do I build a car?

You can build your car using the DeepRacer SDK.

The SDK is a set of tools and libraries that allow you to build and deploy your car to the DeepRacer platform.

How do I deploy my car?

Once you have built your car, you can deploy it to the DeepRacer platform by following the instructions in the Deploying Your Car Guide.

How do I race my car?

Once you have deployed your car to the DeepRacer platform, you can race it by following the instructions in the Racing Your Car Guide.

How do I interact with the DeepRacer platform?

The DeepRacer platform is a cloud-based platform that allows you to interact with your cars and other racers.

You can interact with the platform by followin

In [ ]:
training_params=TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="adamw_bnb_8bit",
    save_steps=25,
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.3,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard")


In [ ]:
trainer=SFTTrainer(
    model="/kaggle/input/gemma/transformers/2b/2",
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1997: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:174: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.604500
200,1.355100
300,1.398000
400,1.349800
500,1.288900
600,1.153000
700,1.109900
800,1.090500
900,1.147300
1000,1.164700


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/2b/2 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/2b/2 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils

TrainOutput(global_step=1018, training_loss=1.2591396324292616, metrics={'train_runtime': 488.3931, 'train_samples_per_second': 2.084, 'train_steps_per_second': 2.084, 'total_flos': 1599304311889920.0, 'train_loss': 1.2591396324292616, 'epoch': 2.0})

In [ ]:
trainer.model.save_pretrained(mew_model)
trainer.tokenizer.save_pretrained(mew_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/gemma/transformers/2b/2 - will assume that the vocabulary was not modified.
  warnings.warn(


('google/gemma-2b-DataScSpeacial/tokenizer_config.json',
 'google/gemma-2b-DataScSpeacial/special_tokens_map.json',
 'google/gemma-2b-DataScSpeacial/tokenizer.model',
 'google/gemma-2b-DataScSpeacial/added_tokens.json',
 'google/gemma-2b-DataScSpeacial/tokenizer.json')

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(mew_model)


model=AutoModelForCausalLM.from_pretrained(mew_model,quantization_config=bnb_config,device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text="what is data science?"
device="cuda:0"
inputs=tokenizer(text,return_tensors="pt").to(device)

outputs=model.generate(**inputs,max_new_tokens=256)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

what is data science? Data Science is the process of using various mathematical and computational techniques to extract meaningful insights from data. It involves analyzing large volumes of data to uncover patterns, draw conclusions, and inform decision-making. Data Science is a rapidly evolving field that combines the disciplines of computer science, statistics, and business analytics to solve real-world problems. It is used in a wide range of industries, including healthcare, finance, marketing, and consumer insights, to inform decision-making and improve decision-making. Data Science involves a rigorous process of data munging, data cleaning, data analysis, and data visualization to extract meaningful insights from data. It involves using advanced algorithms and techniques to uncover patterns, draw conclusions, and inform decision-making. Data Science is a powerful tool for unlocking the hidden insights that exist in data. It is used to inform decision-making, improve business perfo